In [12]:
import pandas as pd
import json

In [13]:
boards = [{1: [1, 2, 3, 4], 2: [4, 3, 2, 1]},
          {1: [1, 2, 3, 4], 2: [3, 4, 1, 2]},
          {1: [1, 2, 3, 4], 2: [2, 1, 4, 3]},
          {1: [1, 2, 3, 4], 2: [1, 2, 3, 4]}]

In [14]:
date = '2018-02-28'

In [59]:
df = pd.DataFrame(
    [{'Day': date,
      'Round': rnd + 1,
      'Board offset': abs((rnd % 2 - 1) + (i // 2)),  # If board offset is even then player 1 is white
      'Team 1 conference': 'Atlantic & Pacific' if division['division'] in ['Atlantic Division', 'Pacific Division'] else 'Central & Eastern',
      'Team 1 division': division['division'],
      'Team 1 full name': f"{divisionList['name'].strip()} {divisionList['surname'].strip()}",
      'Board 1': boards[rnd][1][i],
      'Player 1': f"{player1['name'].strip()}",
      'Rating 1': int(f"{player1['rating'].strip()}"),
      'Team 2 conference': 'Atlantic & Pacific' if division['division'] in ['Atlantic Division', 'Pacific Division']  else 'Central & Eastern',
      'Team 2 division': division['division'],
      'Team 2 full name': f"{divisionList['name2'].strip()} {divisionList['surname2'].strip()}",
      'Board 2': boards[rnd][2][i],
      'Player 2': f"{player2['name'].strip()}",
      'Rating 2': int(f"{player2['rating'].strip()}")
     }
     for division in json.load(open(f'{date}.json'))
     for divisionList in division['divisionList']
     for rnd, rounds in enumerate(divisionList['rounds'])
     for i, (player1, player2) in enumerate(zip(rounds['team1'], rounds['team2']))])

In [60]:
from elo import Elo
elo = Elo(0.39266867, 0.46936407)

In [61]:
def score(x):
    # Player 1 is white on board 1 and 3 (board offset 0 and 2)
    p = x['Board offset'] % 2 == 0
    white_elo = x['Rating 1'] if p else x['Rating 2']
    black_elo = x['Rating 2'] if p else x['Rating 1']
    pr = elo.probabilities(white_elo, black_elo)
    return pd.Series({'Player 1 score': (pr['white'] if p else pr['black']) + pr['draw'] / 2,
                      'Player 2 score': (pr['black'] if p else pr['white']) + pr['draw'] / 2})

score_df = pd.concat([df, df.apply(score, axis=1)], axis=1)

In [62]:
# Sum score by player (+ day, division, board, team)
def f(i):
    return lambda x: pd.Series(
        {'Day': x['Day'],
         'Conference': x[f"Team {i} conference"],
         'Division': x[f'Team {i} division'],
         'Matchup': '{Team 1 full name} v {Team 2 full name}'.format(**x),
         'Board': x[f'Board {i}'],
         'Team': x[f'Team {i} full name'],
         'Player': x[f'Player {i}'],
         'Rating': x[f'Rating {i}'],
         'Score': x[f'Player {i} score']})    

total_df = pd.concat([score_df.apply(f(1), axis=1), score_df.apply(f(2), axis=1)]) \
    .groupby(['Day', 'Conference', 'Division', 'Matchup', 'Board', 'Team', 'Player']) \
    .agg({'Rating': 'mean', 'Score': 'sum'})

In [66]:
# Likely match scores
total_df.groupby(['Day', 'Division', 'Matchup', 'Team'], group_keys=False).agg({'Rating': 'mean', 'Score': 'sum'})

Rating  \
Day        Division          Matchup                                       Team                                
2018-02-28 Atlantic Division Buenos Aires Krakens v Montreal Chessbrahs    Buenos Aires Krakens      2485.00   
                                                                           Montreal Chessbrahs       2520.00   
                             Minnesota Blizzard v Miami Champions          Miami Champions           2440.25   
                                                                           Minnesota Blizzard        2462.50   
                             Montclair Sopranos v Saint Louis Arch Bishops Montclair Sopranos        2487.25   
                                                                           Saint Louis Arch Bishops  2516.25   
                             Webster Windmills v Pittsburgh Pawngrabbers   Pittsburgh Pawngrabbers   2457.50   
                                                                           Webster Windmills         2491.25   
           Central Division  Amsterdam Mosquitoes v Marseille Migraines    Amsterdam Mosquitoes      2428.25   
                                                                           Marseille Migraines       2486.75   
                             Cannes Blockbusters v Ljubljana Turtles       Cannes Blockbusters       2500.00   
                                                                           Ljubljana Turtles         2461.50   
                             London Towers v Stockholm Snowballs           London Towers             2388.25   
                                                                           Stockholm Snowballs       2511.50   
                             Reykjavik Puffins v London Lions              London Lions              2431.00   
                                                                           Reykjavik Puffins         2472.00   
           Eastern Division  Delhi Dynamite v Norway Gnomes                Delhi Dynamite            2477.75   
                                                                           Norway Gnomes             2480.75   
                             Estonia Horses v Armenia Eagles               Armenia Eagles            2495.75   
                                                                           Estonia Horses            2476.00   
                             Mumbai Movers v Riga Magicians                Mumbai Movers             2500.75   
                                                                           Riga Magicians            2485.75   
                             Oslo Trolls v Volga Stormbringers             Oslo Trolls               2439.25   
                                                                           Volga Stormbringers       2499.75   
           Pacific Division  Dallas Destiny v Chengdu Pandas               Chengdu Pandas            2531.75   
                                                                           Dallas Destiny            2518.50   
                             Rio Grande Ospreys v Las Vegas Desert Rats    Las Vegas Desert Rats     2425.50   
                                                                           Rio Grande Ospreys        2496.50   
                             San Diego Surfers v Australia Kangaroos       Australia Kangaroos       2496.25   
                                                                           San Diego Surfers         2450.25   
                             Seattle Sluggers v San Jose Hackers           San Jose Hackers          2483.25   
                                                                           Seattle Sluggers          2519.25   

                                                                                                        Score  
Day        Division          Matchup                                       Team                                
2018-02-28 Atlantic Division Buenos Aires Krakens v Montreal Chessbra

In [73]:
# Top scoring players per board
total_df.groupby(level=['Day', 'Division', 'Board'], group_keys=False).apply(lambda x: x.sort_values('Score', ascending=False).head(1))

Rating  \
Day        Conference         Division          Matchup                                       Board Team                     Player                           
2018-02-28 Atlantic & Pacific Atlantic Division Montclair Sopranos v Saint Louis Arch Bishops 1     Saint Louis Arch Bishops Fabiano Caruana           2799   
                                                                                              2     Saint Louis Arch Bishops Varuzhan Akobian          2662   
                                                                                              3     Saint Louis Arch Bishops Yaroslav Zherebukh        2621   
                                                Minnesota Blizzard v Miami Champions          4     Minnesota Blizzard       Sean Nagle                2432   
           Central & Eastern  Central Division  Amsterdam Mosquitoes v Marseille Migraines    1     Marseille Migraines      Maxime Vachier-Lagrave    2804   
                                                                                              2     Marseille Migraines      Sebastien Maze            2614   
                                                                                              3     Marseille Migraines      Yannick Gozzoli           2582   
                                                Reykjavik Puffins v London Lions              4     Reykjavik Puffins        David Kjartansson         2386   
                              Eastern Division  Delhi Dynamite v Norway Gnomes                1     Norway Gnomes            Magnus Carlsen            2827   
                                                                                              2     Norway Gnomes            Nils Grandelius           2653   
                                                Estonia Horses v Armenia Eagles               3     Armenia Eagles           Karen Grigoryan           2568   
                                                Oslo Trolls v Volga Stormbringers             4     Volga Stormbringers      Mikhail Gorozhanin        2421   
           Atlantic & Pacific Pacific Division  Seattle Sluggers v San Jose Hackers           1     Seattle Sluggers         Hikaru Nakamura           2781   
                                                Dallas Destiny v Chengdu Pandas               2     Chengdu Pandas           Li Chao                   2745   
                                                San Diego Surfers v Australia Kangaroos       3     Australia Kangaroos      Moulthun Ly               2486   
                                                                                              4     Australia Kangaroos      Temur Kuybokarov          2468   

                                                                                                                                                        Score  
Day        Conference         Division          Matchup                                       Board Team                     Player                            
2018-02-28 Atlantic & Pacific Atlantic Division Montclair Sopranos v Saint Louis Arch Bishops 1     Saint Louis Arch Bishops Fabiano Caruana         3.418224  
                                                                                              2     Saint Louis Arch Bishops Varuzhan Akobian        2.920576  
                                                                                              3     Saint Louis Arch Bishops Yaroslav Zherebukh      2.693606  
                                                Minnesota Blizzard v Miami Champions          4     Minnesota Blizzard       Sean Nagle              1.972745  
           Central & Eastern  Central Division  Amsterdam Mosquitoes v Marseille Migraines    1     Marseille Migraines      Maxime Vachier-Lagrave  3.435783  
                                                                                              2     Marseille Migraines      Sebastien Maze          2.911725  
      

In [78]:
# Expected result from model
total_df.groupby(level=['Day', 'Division', 'Board'], group_keys=False) \
    .apply(lambda x: x.sort_values('Score', ascending=False).head(1)) \
    .agg({'Rating': 'mean', 'Score': 'sum'})

Rating    2615.56250
Score       43.10215
dtype: float64

In [70]:
# Top players by rating
total_df.groupby(level=['Day', 'Division', 'Board'], group_keys=False).apply(lambda x: x.sort_values('Rating', ascending=False).head(1))

Rating  \
Day        Conference         Division          Matchup                                       Board Team                     Player                           
2018-02-28 Atlantic & Pacific Atlantic Division Montclair Sopranos v Saint Louis Arch Bishops 1     Saint Louis Arch Bishops Fabiano Caruana           2799   
                                                                                              2     Saint Louis Arch Bishops Varuzhan Akobian          2662   
                                                                                              3     Saint Louis Arch Bishops Yaroslav Zherebukh        2621   
                                                Minnesota Blizzard v Miami Champions          4     Minnesota Blizzard       Sean Nagle                2432   
           Central & Eastern  Central Division  Amsterdam Mosquitoes v Marseille Migraines    1     Marseille Migraines      Maxime Vachier-Lagrave    2804   
                                                                                              2     Marseille Migraines      Sebastien Maze            2614   
                                                                                              3     Marseille Migraines      Yannick Gozzoli           2582   
                                                Reykjavik Puffins v London Lions              4     Reykjavik Puffins        David Kjartansson         2386   
                              Eastern Division  Delhi Dynamite v Norway Gnomes                1     Norway Gnomes            Magnus Carlsen            2827   
                                                Mumbai Movers v Riga Magicians                2     Mumbai Movers            Surya Shekhar Ganguly     2665   
                                                Estonia Horses v Armenia Eagles               3     Armenia Eagles           Karen Grigoryan           2568   
                                                Oslo Trolls v Volga Stormbringers             4     Volga Stormbringers      Mikhail Gorozhanin        2421   
           Atlantic & Pacific Pacific Division  Seattle Sluggers v San Jose Hackers           1     Seattle Sluggers         Hikaru Nakamura           2781   
                                                Dallas Destiny v Chengdu Pandas               2     Chengdu Pandas           Li Chao                   2745   
                                                                                              3     Dallas Destiny           Elshan Moradiabadi        2507   
                                                San Diego Surfers v Australia Kangaroos       4     Australia Kangaroos      Temur Kuybokarov          2468   

                                                                                                                                                        Score  
Day        Conference         Division          Matchup                                       Board Team                     Player                            
2018-02-28 Atlantic & Pacific Atlantic Division Montclair Sopranos v Saint Louis Arch Bishops 1     Saint Louis Arch Bishops Fabiano Caruana         3.418224  
                                                                                              2     Saint Louis Arch Bishops Varuzhan Akobian        2.920576  
                                                                                              3     Saint Louis Arch Bishops Yaroslav Zherebukh      2.693606  
                                                Minnesota Blizzard v Miami Champions          4     Minnesota Blizzard       Sean Nagle              1.972745  
           Central & Eastern  Central Division  Amsterdam Mosquitoes v Marseille Migraines    1     Marseille Migraines      Maxime Vachier-Lagrave  3.435783  
                                                                                              2     Marseille Migraines      Sebastien Maze          2.911725  
      

In [71]:
# Tie breaker
total_df.groupby(['Day', 'Team']).agg({'Rating': 'mean', 'Score': 'sum'}).sort_values('Score', ascending=False)

Rating     Score
Day        Team                                       
2018-02-28 Stockholm Snowballs       2511.50  9.950843
           Rio Grande Ospreys        2496.50  9.352237
           Marseille Migraines       2486.75  9.335120
           Volga Stormbringers       2499.75  9.225419
           Saint Louis Arch Bishops  2516.25  9.195436
           Reykjavik Puffins         2472.00  8.887307
           Montreal Chessbrahs       2520.00  8.688907
           Australia Kangaroos       2496.25  8.585984
           Webster Windmills         2491.25  8.535698
           Seattle Sluggers          2519.25  8.528273
           Armenia Eagles            2495.75  8.462987
           Chengdu Pandas            2531.75  8.453046
           Mumbai Movers             2500.75  8.415382
           Cannes Blockbusters       2500.00  8.339484
           Minnesota Blizzard        2462.50  8.326212
           Norway Gnomes             2480.75  8.278113
           Delhi Dynamite            2477.75  7.721887
           Miami Champions           2440.25  7.673788
           Ljubljana Turtles         2461.50  7.660516
           Riga Magicians            2485.75  7.584618
           Dallas Destiny            2518.50  7.546954
           Estonia Horses            2476.00  7.537013
           San Jose Hackers          2483.25  7.471727
           Pittsburgh Pawngrabbers   2457.50  7.464302
           San Diego Surfers         2450.25  7.414016
           Buenos Aires Krakens      2485.00  7.311093
           London Lions              2431.00  7.112693
           Montclair Sopranos        2487.25  6.804564
           Oslo Trolls               2439.25  6.774581
           Amsterdam Mosquitoes      2428.25  6.664880
           Las Vegas Desert Rats     2425.50  6.647763
           London Towers             2388.25  6.049157